<a href="https://colab.research.google.com/github/SDHAMEED/FMML_PROJECTS_AND_LABS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [9]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-9-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [10]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [11]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [12]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [13]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [14]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [15]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [16]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [17]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [18]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [19]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [20]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [21]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1 ANS)
Yes, averaging validation accuracy across multiple splits can provide more consistent and reliable results when evaluating machine learning models, especially in situations where there is randomness involved, such as with different random initializations of the model or the random shuffling of data during the split. This technique is known as cross-validation.


1)Reduces Variance: By splitting your dataset into multiple subsets (folds) and evaluating the model's performance on each fold separately, you obtain a set of accuracy values. Averaging these values reduces the influence of random fluctuations in a single split, providing a more stable estimate of your model's performance.

2)Better Generalization: Averaging over multiple splits helps you assess how well your model generalizes to different subsets of your data. If your model performs consistently well across all folds, it's more likely to generalize well to unseen data.

3)Mitigates Data Split Bias: In some cases, a single random split of your data might lead to a particularly favorable or unfavorable set of training and validation examples. Averaging over multiple splits helps mitigate the impact of this bias.

2ans)

The accuracy of a test or measurement depends on various factors, including the methodology used, the quality of data, and the specific context in which it is applied. In the context of machine learning and model evaluation, "test accuracy" typically refers to how well a machine learning model performs on a test dataset that it hasn't seen during training. The accuracy of this estimate depends on several factors:

Size and Representativeness of the Test Dataset: A larger and more representative test dataset generally provides a more accurate estimate of a model's performance. If the test dataset is too small or not representative of the real-world data the model will encounter, the estimate may not be reliable.

Quality of Labels: If the labels in the test dataset are noisy or incorrect, it can lead to inaccurate estimates of model performance. Ensuring high-quality labels is crucial.

Model Complexity: The complexity of the model being evaluated can also affect the accuracy of the estimate. Overly complex models may overfit the training data and perform poorly on unseen data, leading to an overestimated test accuracy.

Data Preprocessing: How the data is preprocessed, including feature engineering and data cleaning, can impact the test accuracy. Inaccurate or inappropriate preprocessing can lead to biased estimates.

Cross-Validation: Techniques like k-fold cross-validation can provide a more robust estimate of model performance by partitioning the data into multiple subsets for training and testing. This helps mitigate the impact of data variability.

Hyperparameter Tuning: The choice of hyperparameters can significantly affect a model's performance. Proper hyperparameter tuning can improve accuracy.

Randomness: Some machine learning algorithms, like random forests or neural networks with random initialization, can produce slightly different results each time they are trained. Running multiple evaluations and averaging the results can provide a more accurate estimate.

Domain-Specific Considerations: In some domains, certain types of errors may be more costly or important than others. Accuracy may not be the best metric, and other metrics like precision, recall, F1 score, or area under the ROC curve may provide a more accurate picture of performance.

In summary, while machine learning models can provide estimates of test accuracy, the accuracy of these estimates depends on multiple factors. To obtain more accurate estimates, it's essential to carefully design experiments, use appropriate evaluation metrics, and ensure data quality and representativeness.








3ans)

The effect of the number of iterations on an estimate depends on the context and the specific estimation or computational algorithm being used. In many cases, increasing the number of iterations can lead to a more accurate estimate, but there are also situations where more iterations do not necessarily result in a better estimate or where there are diminishing returns with additional iterations.

Here are a few scenarios to consider:

Numerical Integration:

In numerical integration methods like the trapezoidal rule or Simpson's rule, increasing the number of iterations (i.e., increasing the number of subintervals) typically leads to a more accurate estimate of the integral of a function.
Optimization Algorithms:

In optimization algorithms like gradient descent, increasing the number of iterations can help the algorithm converge to a better solution. However, there may be a point of diminishing returns where further iterations do not significantly improve the solution.
Monte Carlo Simulation:

In Monte Carlo simulations, more iterations (i.e., more random samples) can lead to a more accurate estimate of a probability or expected value. The law of large numbers suggests that as the number of iterations approaches infinity, the estimate becomes more accurate.
Machine Learning Training:

In machine learning, training algorithms often involve iterative processes such as gradient descent. Increasing the number of iterations during training can improve the model's performance, up to a certain point. After a certain number of iterations, the model may start overfitting the training data and perform worse on new, unseen data.
Convergence:

Some iterative algorithms have convergence criteria. When a specific convergence criterion is met, the algorithm terminates. Increasing the number of iterations beyond this point may not improve the estimate.
In summary, the effect of the number of iterations on an estimate depends on the specific problem, algorithm, and the convergence behavior of that algorithm. While more iterations often lead to better estimates, there can be situations where additional iterations do not provide significant improvements, and it's essential to consider computational resources, convergence criteria, and the problem's nature when determining the appropriate number of iterations.







4ans)

Dealing with a very small training dataset or validation dataset by increasing the number of iterations (epochs) can have both benefits and limitations. Let's examine the implications:

Benefits:

Overfitting Mitigation: Increasing the number of iterations may help the model converge better on the small training dataset. It can allow the model to see the limited data more times and potentially reduce overfitting.

Performance Improvement: In some cases, a model trained on a small dataset may improve its performance as it learns more about the available data through additional iterations.

Limitations:

Data Limitation: No matter how many iterations you run, the model can only learn from the data it has. If the dataset is too small or not representative of the underlying distribution, the model's performance may still be limited. Increasing iterations won't create information that doesn't exist in the data.

Computational Cost: Training deep learning models for a large number of iterations can be computationally expensive, especially for complex models. It may not be a cost-effective solution, and you might reach diminishing returns in terms of performance improvement.

Overfitting Risk: While more iterations can help reduce overfitting to some extent, there's a risk of overfitting to the small dataset even more if the model becomes too complex. Regularization techniques like dropout or weight decay might be necessary to counter this risk.

Validation Dataset: If you also have a small validation dataset, it may not be representative enough to accurately assess model performance, even with increased iterations. Validation data helps you understand how well the model generalizes to unseen data, and a small validation set can lead to unreliable evaluation.

In summary, increasing the number of iterations can be a useful strategy for training models on small datasets, but it's not a magic solution. It should be done carefully while monitoring for overfitting, and it may not fully address the limitations imposed by a small dataset, especially if the data is not representative of the problem domain. It's also important to have a sufficient validation dataset to reliably evaluate model performance.





